# MongoDB Aggregation Pipeline In Python

Learn about the various stages and configurations you can create to configure an Aggregation Pipeline

Stages: [match](#match), [project](#project), [unset](#unset), [limit](#limit), [skip](#skip), [sort](#sort), [count](#count), [sortByCount](#sortByCount), [unwind](#unwind), [group](#group), [addFields](#addFields), [sample](#sample), [lookup](#lookup), [unionWith](#unionWith), [out](#out), [merge](#merge)

Operators: [size](#size-(operator)), [in](#in-(operator)), [arrayElemAt](#arrayElemAt-(operator)), [first](#first-(operator)), [count](#count-(accumulator-operator)), [sum](#sum-(accumulator-operator)), [first, last](#first,-last-(accumulator-operators)), [push](#push-(accumulator-operator)), [addToSet](#addToSet-(accumulator-operator)), [regexMatch](#regexMatch-(operator)), [cond](#cond-(operator)), [Date](#Date-Operators), [expr](#expr-(operator)), [ifNull](#ifNull-(operator)), [type](#type-(operator)), [switch](#switch-(operator))

In [1]:
from pymongo import MongoClient

In [2]:
mongodb_uri = "mongodb://localhost:27017/"
db_name = "aggregation_test"
client = MongoClient(mongodb_uri)
db = client[db_name]

### Helper Function

In [3]:
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

### Inserting Some Sample Data

In [4]:
import insert_aggregation_sample_data as iasd
iasd.insert_data(mongodb_uri, db_name)

Done


### match

### project

### unset

### limit

### skip

### sort

### count

### sortByCount

### size (operator)

### in (operator)

### arrayElemAt (operator)

### first (operator)

### unwind

### group

### count (accumulator operator)

### sum (accumulator operator)

### first, last (accumulator operators)

### push (accumulator operator)

### addToSet (accumulator operator)

### $$ROOT (system variable)

### addFields

### sample

### lookup

### unionWith

### regexMatch (operator)

### out
*Note: You can potentially overwrite all your data in a collection with this stage, use with caution*

### merge
*Note: You can potentially overwrite data within a collection with this stage, use with caution*

### cond (operator)

### $$NOW (system variable)

### Date Operators

### expr (operator)

### ifNull (operator)

### type (operator)

### switch (operator)